<a href="https://colab.research.google.com/github/57005/Chatbot-using-Llama2/blob/main/Agent_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai
!pip install duckduckgo_search
!pip install dotenv
!pip install openai
!pip install crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.7/194.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
import os
from crewai import Agent
from crewai import Task
from crewai import Process,Crew
from crewai_tools import BaseTool
from duckduckgo_search import DDGS

In [ ]:
from dotenv import load_dotenv

In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
load_dotenv()

False

In [ ]:
os.environ['OPENAI_MODEL_NAME']= 'gpt-3.5-turbo'

In [ ]:
class NewsSearchTool(BaseTool):
  name:str= 'news_search'
  description:str= 'search news on the given topic from internet and provide results'

  def _run(self, topic : str) -> str:
    ddgs = DDGS()
    results = ddgs.news(keywords=topic, timelimit="d", max_results=20)
    result_string=''

    for result in results:
      result_string += f"Title:{result['title']}\n Description: {result['body']}\n\n"

    if len(result_string.strip()) < 2:
      result_string = 'No news found for given keyword'
      return result_string.strip()


In [ ]:
newssearchtool = NewsSearchTool()

In [ ]:
## Creating a researcher agent with memory and verbose mode
researcher = Agent(
    role= "Researcher",
    goal='Get the latest news in {topic}',
    verbose = True,
    memory = True,
    backstory=(
       " As a knowledge enthusiast, you always look for new happenings on a given topic "
      "and share the knowledge with broader community."
    ),
    tools=[newssearchtool],
)

In [ ]:
# Creating a writer agent with custom tools and delegation capability
summarizer = Agent(
    role='Summarizer',
    goal='Summarize the news in {topic}',
    verbose=True,
    memory= True,
    backstory= (
       "With a flair for simplifying complex topics, you "
    "remove redundant information and summarize the "
    "discoveries in an accessible manner."
    ),
    allow_delegation = False
    )

In [ ]:
#Research task

research_task = Task(
  description=(
    "Get the latest news in {topic}."
  ),
  expected_output='Points covering the latest news on {topic}.',
  tools=[newssearchtool],
  agent=researcher,
)


In [ ]:
#summarize the task

summarize_task = Task(

    description = (
        "Given all the news about the {topic}, summarize them to 2 or 3 bullet points. Discard the news which do not have central focus on {topic}."
      "If no news is relevant, return that no news found for the {topic}"
    ),
    expected_output = '2 or 3 summarized bullet points, each one focussing on one specific news about the {topic}. If no news found, return no news found',
    agent = summarizer,
    output_file = 'news_summary.txt',
)

In [ ]:
# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
    agents=[researcher, summarizer],
    tasks=[research_task, summarize_task],
    process=Process.sequential,
    verbose=True,

)

In [ ]:
result = crew.kickoff(inputs ={'topic': 'HDFC Bank'})
print(result)

 [2024-06-28 14:26:08][DEBUG]: == Working Agent: Researcher
 [2024-06-28 14:26:08][INFO]: == Starting Task: Get the latest news in HDFC Bank.


> Entering new CrewAgentExecutor chain...
I need to use the tools available to gather the latest news on HDFC Bank.

Action:
Delegate work to coworker

Action Input:
{
    "task": "Search for the latest news on HDFC Bank",
    "context": "I am a researcher looking to gather the latest news on HDFC Bank for knowledge sharing purposes."
}
 


Error executing tool. coworker mentioned not found, it must be one of the following options:
- summarizer


Action: Delegate work to coworker

Action Input:
{
    "task": "Search for the latest news on HDFC Bank",
    "context": "I am a researcher looking to gather the latest news on HDFC Bank for knowledge sharing purposes.",
    "coworker": "Summarizer"
}

> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
The latest news on HDFC Bank includes the announcement of their p